## load config module

Need to go this path because my config files are not stored in the same directory as the project code.

In [21]:
# get config data
import os
import sys

configfile = '~/projects/config/binance/config.py'
sys.path.append(os.path.dirname(os.path.expanduser(configfile)))

import config
print(f"Loaded KEY: {config.API_KEY[:5]}..., and  SECRECT: {config.API_SECRET[:5]}... .")

Loaded KEY: J2EDR..., and  SECRECT: fERKk... .
